In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as pl
import wradlib
from osgeo import osr
import datetime as dt
import glob
from osgeo import gdal
from scipy.spatial import cKDTree

% matplotlib inline 

#from pcc import get_time_of_gpm
import datetime as dt

### Used Functions
-----------------

In [2]:
def dpr_swath_contour(sr_grid):
    """

    Function for dpr swath grid contour
    
    ## sr_grid[:, 0]        # - complete left column
    ## sr_grid[-1,1:-1]     # - upper edge without left and right corner
    ## sr_grid[:, -1][::-1] # - complete right column (backward)
    ## sr_grid[0,0:-1]      # - lower edge (without right corner)
    
    Input:
    ------
    sr_grid ::: Space-borne grid x,y
    
    Output:
    -------
    trg ::: Contour of the space-borne grid
    
    """
    
    trg = np.r_[sr_grid[:, 0], sr_grid[-1, 1:-1], sr_grid[:, -1][::-1], sr_grid[0, 0:-1][::-1]]
    
    return trg


In [3]:
def idx_sr_in_gr_area(_bingrid, _gpm_xy):
    """
    Funktion:
    ---------
    Search for SR Footprint (Index) that are located in the GR scan area 
    
    Input: 
    ------
    _bingrid ::: Binary-grid of the GR scan area
        
    _gpm_xy ::: GPM footptint coordinates
    
    Output:
    -------
    
    _gpm_xy_outer_idx ::: index of all footprints not included in radolan scan area
    _gpm_xy_inner_idx ::: idex of all footprints included in radolan area but
                           not scanned by ground radars 
    
    """
    _gpm_xy = _gpm_xy.copy()
    # Bestimmen von Eckpunkten bei RADOLAN RY GRID!
    # HARDCODED: but ok for regula RADOLAN Produkts
    xmin, xmax = -523.4621669218559, 375.5378330781441
    ymin, ymax = -4658.644724265572, -3759.644724265572
    
    # Radolangitter um 1 gridpoit zu allen seiten erweitern
    _xex, _yex = np.arange(xmin-1, xmax+2, 1), np.arange(ymin-1, ymax+2, 1)
    _xxx, _yyy = np.meshgrid(_xex,_yex)
    
    # Bin grid bestimmen
    _bingrid = _bingrid.copy()
    _bingrid[_bingrid >= 0]=1
    _bingrid[_bingrid < 0]=0
    
    # Gittererweiterungen zusammenstellen
    ## rn_tb ::: bin gird top and bottom
    ## rn_lr ::: bin gird left and right
    rn_tb = np.zeros(900)
    rn_lr = np.zeros(902)
    
    # Griderweiterungen einsetzen
    # erst oben und unten
    rn1 = np.c_[rn_tb, _bingrid, rn_tb]
    # dann rechts
    rn2 = np.vstack((rn1, rn_lr))
    # dann rechts
    rn3 = np.vstack((rn_lr,rn2))
    
    from skimage import measure
    # Contouren des Bin-grids erstellen
    contours = measure.find_contours(rn3, 0, positive_orientation='high',
                                     fully_connected='high' )
    ### print(len(contours), ' detected Polygons')
    #print("---------------------")
    #for j in range(len(contours)):
    #    print(len(contours[j]))
    #print("---------------------")

    # bestiimung der Listen für Polygon Points
    _xx = []
    _yy = []

    # suche nach dem größten Polygon... dieser sollte der RADOLAN UMRISS sein
    for i in range(len(contours)):
        # Polygone mit 3 oder weniger points ergeben keine nutzbare Fläche
        if len(_xxx[contours[i][:,0].astype(int),contours[i][:,1].astype(int)]) <= 3:
            print('not relevant polygon removed')
        else:   
            # PolygonPoints bestimmen (RADOLAN RAND POINTS)
            _xx.append(_xxx[contours[i][:,0].astype(int),contours[i][:,1].astype(int)])
            _yy.append(_yyy[contours[i][:,0].astype(int),contours[i][:,1].astype(int)])
            
    # sortiern nach der länge, das letzt element ist das größte und sollte somit der Radolarand sein
    _xx.sort(key=len)
    _yy.sort(key=len)

    # Outter polygon erstellen
    _xy = np.vstack((_xx[-1].ravel(), _yy[-1].ravel())).transpose()    
    
    # Suche nach den GPM footprints im Scanngebiet von RADOLAN mit ZONALSTATS
    zdpoly = wradlib.zonalstats.ZonalDataPoly(_gpm_xy, [_xy])
    _gpm_xy_outer_idx = zdpoly.get_source_index(0)
    
    # Array for inner Polygons
    _gpm_xy_inner_idx = np.array([])
    
    # Entferne Inner Polygons
    # Auch nur wenn es weitere Polygone gibt 
    if len(_xx)>1:

        for inner_poly_index in range(len(_xx)-1):

            ### print ('Polygon size: ', len(_xx[inner_poly_index].ravel()))

            _xy_inner = np.vstack((_xx[inner_poly_index].ravel(),
                                  _yy[inner_poly_index].ravel())).transpose()

            _zdpoly = wradlib.zonalstats.ZonalDataPoly(_gpm_xy, [_xy_inner])

            _inner_idx = _zdpoly.get_source_index(0)

            _gpm_xy_inner_idx = np.append(_gpm_xy_inner_idx, _inner_idx)


            ### if _gpm_xy_inner_idx.size==0:
                ### print ('inner polygons do not match with SR grid')

            ### else:
                ### print('match idx with inner polygon: ', _gpm_xy_inner_idx.shape)
                    

    
    _idx_r = ~np.isin(_gpm_xy_outer_idx, _gpm_xy_inner_idx)
    
    return _gpm_xy_outer_idx[_idx_r]
        

In [4]:
def dpr_antenna_weighting(r):
    """
    Funktion: explained in 
    Watters, D., A. Battaglia, K. Mroz, and F. Tridon, 0: 
    Validation of the GPM Version-5 Surface Rainfall 
    Products over Great Britain and Ireland. 
    J. Hydrometeor., 0, https://doi.org/10.1175/JHM-D-18-0051.1
    
    and
    
    Mroz, K., A. Battaglia, T. J. Lang, D. J. Cecil, 
    S. Tanelli, and F. Tridon, 2017:  Hail-detection
    algorithm for the GPM Core Observatory satellite sensors.
    Journal of Applied Meteorology and Climatology,56 (7), 1939–1957
    
    Input:
    ------
    r ::: Distance to the center point of the footprint
    
    Output:
    -------
    omega ::: Weighting for the point with the distance
    
    """
    
    omega = np.exp(-(r/2.5)**2. * np.log10(4.))
    
    return omega

In [5]:
def ipoli_radi_w(xy_cut, gpm_xy,rwdata_cut, dpr_footprint, k=25):
    """
    ...
    """
    # cKDTree radolan
    tree = cKDTree(xy_cut, balanced_tree=False)

    # cKDTree gpm dpr
    tree_gpm = cKDTree(gpm_xy, balanced_tree=True)

    dists, ix = tree.query(gpm_xy, k=k, n_jobs=-1) # k maximal possible ry pixel in dpr footprint
    ix2 = tree.query_ball_point(gpm_xy, dpr_footprint)

    ry_pns_w = []

    for i in range(ix.shape[0]):
        #i is all points in one dpr footprint
        #distancen for all i
        index = np.isin(ix[i,:],ix2[i] )
        res1 = np.nansum( dpr_antenna_weighting(dists[i,:][index]) * rwdata_cut.ravel()[ix[i,:][index]])
        res2 = np.nansum(dpr_antenna_weighting(dists[i,:][index]))

        ry_pns_w.append(res1/res2)


    return np.array(ry_pns_w)   

In [6]:
def ipoli_radi(xy_cut, gpm_xy,rwdata_cut, dpr_footprint, k=25, calc='mean'):
    """
    calc: mean, max, min
    """
    # cKDTree radolan
    tree = cKDTree(xy_cut, balanced_tree=False)

    # cKDTree gpm dpr
    tree_gpm = cKDTree(gpm_xy, balanced_tree=True)

    dists, ix = tree.query(gpm_xy, k=k, n_jobs=-1) # k maximal possible ry pixel in dpr footprint
    ix2 = tree.query_ball_point(gpm_xy, dpr_footprint)

    ry_par = []
    
    if calc=='mean':

        for i in range(ix.shape[0]):
            #i is all points in one dpr footprint
            #distancen for all i
            index = np.isin(ix[i,:],ix2[i] )
            res1 = np.nanmean(rwdata_cut.ravel()[ix[i,:][index]])
            ry_par.append(res1)
            
    elif calc=='max':

            for i in range(ix.shape[0]):
                #i is all points in one dpr footprint
                #distancen for all i
                index = np.isin(ix[i,:],ix2[i] )
                res1 = np.nanmax(rwdata_cut.ravel()[ix[i,:][index]])
                ry_par.append(res1)
              
    elif calc=='min':

            for i in range(ix.shape[0]):
                #i is all points in one dpr footprint
                #distancen for all i
                index = np.isin(ix[i,:],ix2[i] )
                res1 = np.nanmin(rwdata_cut.ravel()[ix[i,:][index]])
                ry_par.append(res1) 
                
    else:
        print ('Wrong calc parameter declaration!')


    return np.array(ry_par)   

In [7]:
def gpm_scan_time(filename, scanswath='NS'):
    from netCDF4 import Dataset
    pr_data = Dataset(filename, mode="r")
    year = pr_data[scanswath]['ScanTime'].variables['Year']
    month = pr_data[scanswath]['ScanTime'].variables['Month']
    dayofmonth = pr_data[scanswath]['ScanTime'].variables['DayOfMonth']
    # dayofyear = pr_data['NS']['ScanTime'].variables['DayOfYear'][mask]
    hour = pr_data[scanswath]['ScanTime'].variables['Hour']
    minute = pr_data[scanswath]['ScanTime'].variables['Minute']
    second = pr_data[scanswath]['ScanTime'].variables['Second']
    # secondofday = pr_data['NS']['ScanTime'].variables['SecondOfDay'][mask]
    millisecond = pr_data[scanswath]['ScanTime'].variables['MilliSecond']
    date_array = zip(year, month, dayofmonth,
                     hour, minute, second,
                     millisecond)
    pr_time = np.array(
        [dt.datetime(d[0], d[1], d[2], d[3], d[4], d[5], d[6]) for d in date_array])
    
    #ttt = gpm_scan_time(gpm_file)
    s_n = {'NS':49, 'HS':24, 'MS':25}
    
    time_array = np.array(s_n[scanswath] * [pr_time]).T
    
    return time_array

In [8]:
def get_time_of_gpm2(gpm_time):
    """einfache mittlere zeit"""
    iii = int(len(np.array(gpm_time['Year']))/2)
    gpm_year = np.array(gpm_time['Year'])[iii]
    gpm_month = np.array(gpm_time['Month'])[iii]
    gpm_day = np.array(gpm_time['DayOfMonth'])[iii]
    gpm_hour = np.array(gpm_time['Hour'])[iii]
    gpm_min = np.array(gpm_time['Minute'])[iii]
    gpm_sek = np.array(gpm_time['Second'])[iii]
    gpm_dt = dt.datetime(gpm_year,gpm_month, gpm_day, gpm_hour, gpm_min, gpm_sek).strftime("%Y.%m.%d -- %H:%M:%S")
    return gpm_dt

In [9]:
def write2hdf(name, x, y, sr_pns, sr_pes, sr_pav, sr_bbh, sr_bbw, sr_phase, sr_type, sr_sth,
             sr_cfb, sr_lst, sr_ele, sr_tim, gr_pp_ipoli, gr_xy, gr_pp, gr_h_max, gr_h_min, gr_h_mean, gr_h_std,
             sr_time, gr_time, sr_aph):
    """
    
    Write data in HDF5 file
    
    """
    hdat = h5py.File(str(name) + '.hdf5', 'w')
    
    ## Create Groups SR and GR estimates
    grp_sr = hdat.create_group("SR")
    grp_sr.attrs['description']=" space-borne estimates (for more information see gpm ATBD)"
    grp_gr = hdat.create_group("GR")
    grp_gr.attrs['description']=" ground-based estimates (for more information see RADOLAN Docs)"
    
    ## SR
    g_x=grp_sr.create_dataset('sr_x', data=x, compression="gzip")
    g_x.attrs['description'] = 'space-borne x-coordinates in km' 
    
    g_y = grp_sr.create_dataset('sr_y', data=y, compression="gzip")
    g_y.attrs['description'] = 'space-borne y-coordinates in km'
    
    g_sr_pns = grp_sr.create_dataset('sr_pns', data=sr_pns, compression="gzip")
    g_sr_pns.attrs['description'] = 'precip near surface (mm/h)'
    
    g_sr_pes = grp_sr.create_dataset('sr_pes', data=sr_pes, compression="gzip")
    g_sr_pes.attrs['description'] = 'adjusted precip estimated observ. height (mm/h)'
    
    g_sr_pav = grp_sr.create_dataset('sr_pav', data=sr_pav, compression="gzip")
    g_sr_pav.attrs['description'] = 'precip mean between 2 and 4 km (mm/h)'
    
    g_sr_bbh = grp_sr.create_dataset('sr_bbh', data=sr_bbh, compression="gzip")
    g_sr_bbh.attrs['description'] = 'bright band height (m)'
    
    g_sr_bbw = grp_sr.create_dataset('sr_bbw', data=sr_bbw, compression="gzip")
    g_sr_bbw.attrs['description'] = 'bright band width (m)'
    
    g_sr_pha = grp_sr.create_dataset('sr_phase', data=sr_phase, compression="gzip")
    g_sr_pha.attrs['description'] = 'hydrometeor phase near surface //100 ->, 0=solid, 1=mixed, 2=liquid'
    
    g_sr_typ = grp_sr.create_dataset('sr_type', data=sr_type, compression="gzip")
    g_sr_typ.attrs['description'] = 'precip type, 1=strat, 2=conv, 3=other'
    
    g_sr_sth = grp_sr.create_dataset('sr_sth', data=sr_sth, compression="gzip")
    g_sr_sth.attrs['description'] = 'storm top hight (m)'
    
    g_sr_cfb = grp_sr.create_dataset('sr_cfb', data=sr_cfb, compression="gzip")
    g_sr_cfb.attrs['description'] = 'range bin number for clutter free bottom'
    
    g_sr_lst = grp_sr.create_dataset('sr_lst', data=sr_lst, compression="gzip")
    g_sr_lst.attrs['description'] = 'land surface type, O-99=Ocean, 100-199=Land,200-299=Coast, 300-399=InlandWater'
    
    g_sr_ele = grp_sr.create_dataset('sr_ele', data=sr_ele, compression="gzip")
    g_sr_ele.attrs['description'] = 'elevation of measurement point, Copy of DEMHmean (m)'
    
    g_sr_times = grp_sr.create_dataset('sr_times', data=sr_tim, compression="gzip")
    g_sr_times.attrs['description'] = 'scan times'
    
    g_sr_aph = grp_sr.create_dataset('sr_aph', data=sr_aph, compression="gzip")
    g_sr_aph.attrs['description'] = 'adjusted hydrometeor phase near surface//100 ->, 0=solid, 1=mixed, 2=liquid'
    
    ## GR
    g_gr_ipp = grp_gr.create_dataset('gr_pp_ipoli', data=gr_pp_ipoli, compression="gzip")
    g_gr_ipp.attrs['description'] = 'ground precip ipol on sr grid'
    g_gr_xyc = grp_gr.create_dataset('gr_xy', data=gr_xy, compression="gzip")
    g_gr_xyc.attrs['description'] = 'ground based xy coordinates (km)'
    g_gr_npp = grp_gr.create_dataset('gr_pp', data=gr_pp, compression="gzip")
    g_gr_npp.attrs['description'] = 'ground precip on gr grid'
    g_gr_hmax = grp_gr.create_dataset('gr_h_max', data=gr_h_max, compression="gzip")
    g_gr_hmax.attrs['description'] = 'max measured height over ground on sr grid (m)'
    g_gr_hmin = grp_gr.create_dataset('gr_h_min', data=gr_h_min, compression="gzip")
    g_gr_hmin.attrs['description'] = 'min measured height over ground on sr grid (m)'
    g_gr_hmea = grp_gr.create_dataset('gr_h_mean', data=gr_h_mean, compression="gzip")
    g_gr_hmea.attrs['description'] = 'mean measured height over ground on sr grid (m)'
    g_gr_hstd = grp_gr.create_dataset('gr_h_std', data=gr_h_std, compression="gzip")
    g_gr_hstd.attrs['description'] = 'std measured height over ground on sr grid (m)'
    
    # Times
    g_sr_otime = grp_sr.create_dataset('sr_time', data=sr_time)
    g_sr_otime.attrs['description'] = 'SR Overpass time'
    g_gr_otime = grp_gr.create_dataset('gr_time', data=gr_time)
    g_gr_otime.attrs['description'] = 'GR Meassured time'

    hdat.close()
    


In [10]:
def calc_height(bin_range_number, elipsoid_offset, localZenithAngle, scan_swath='NS'):
    # Function for height calculation from bins.....
    # GPM DPR ATBD Level 2 2017 Awaka et al S. 21

    if scan_swath in ('NS', 'MS'):
        rangeBinSize = 125.
        binEllipsoid = 176.
    else:
        rangeBinSize = 250.
        binEllipsoid = 88.

    calc_1 = ((binEllipsoid - bin_range_number) * rangeBinSize + elipsoid_offset)

    calc_h = calc_1  * np.cos(np.deg2rad(localZenithAngle))

    return calc_h



def calc_bin(height, elipsoid_offset, localZenithAngle, scan_swath='NS'):

    # Function for height calculation from bins.....
    # GPM DPR ATBD Level 2 2017 Awaka et al S. 21

    if scan_swath in ('NS', 'MS'):
        rangeBinSize = 125.
        binEllipsoid = 176.
    else:
        rangeBinSize = 250.
        binEllipsoid = 88.

    res0 = (height/np.cos(np.deg2rad(localZenithAngle))) - elipsoid_offset 
    res1 = res0 / rangeBinSize 
    res_bin = binEllipsoid - res1

    return res_bin

### Import other parameters


In [11]:
%%time

# Import radolan grid src
radolan_xy = wradlib.georef.get_radolan_grid(900,900)
radolan_xy = radolan_xy.reshape(-1, radolan_xy.shape[-1])
zd = wradlib.zonalstats.DataSource(radolan_xy, name='src')


# Determine projection
proj_stereo = wradlib.georef.create_osr("dwd-radolan")
proj_wgs = osr.SpatialReference()
proj_wgs.ImportFromEPSG(4326)

# Import RADOLAN Heights
rhmax = np.load("/automount/ags/velibor/data/radolan_dx/RADOLAN_H_xymax_900x900_dem1x1.npy")[2]
rhmin = np.load("/automount/ags/velibor/data/radolan_dx/RADOLAN_H_xymin_900x900_dem1x1.npy")[2]
rhmean = np.load("/automount/ags/velibor/data/radolan_dx/RADOLAN_H_xymean_900x900_dem1x1.npy")[2]
rhstd = np.load("/automount/ags/velibor/data/radolan_dx/RADOLAN_H_xystd_900x900.npy")[2]


# Other Parameters
# dpr_footprint
# dpr swath NS MS HS
# Scan swath
sc = 'NS'
dpr_para = {'NS':176, 'HS':88, 'MS':176}


CPU times: user 18.1 s, sys: 652 ms, total: 18.8 s
Wall time: 18.2 s


In [14]:
#%%time

gpm_dir =sorted(glob.glob('/automount/ags/velibor/gpmdata/dprV7_small/*radolan.GPM*.HDF5'))
gpm_dir = gpm_dir[200:1003]
gpm_dir = gpm_dir[::-1]

for gpm_file in gpm_dir:
    # read gpm file with importatnt parameters
    gpmdpr = h5py.File(gpm_file, 'r')

    # gpm scantimes
    gpm_scan_times = gpm_scan_time(gpm_file, scanswath=sc)

    # GPM navigation and time
    gpm_lat = np.array(gpmdpr[sc]['Latitude'])
    gpm_lon = np.array(gpmdpr[sc]['Longitude'])
    gpm_time = gpmdpr[sc]['ScanTime']


    try:
        #gpm_zeit = get_time_of_gpm(gpm_lon, gpm_lat, gpm_time)
        gpm_zeit = get_time_of_gpm2(gpm_time)

    except ValueError:
        pass
        print ('____________ValueError____________')
    else:
        #gpm_zeit = get_time_of_gpm(gpm_lon, gpm_lat, gpm_time)
        print gpm_zeit
        ht, mt = gpm_zeit[14:16], str(int(round(float(gpm_zeit[17:19])/5.0)*5.0))
        year, ye, m, d = gpm_zeit[0:4], gpm_zeit[2:4], gpm_zeit[5:7], gpm_zeit[8:10]

        if mt == '0':
            mt = '00'
        if mt == '5':
            mt = '05'
        if mt == '60':
            mt = '55'
            ht = str(int(ht)+1)
            if ht == '24':
                d = str(int(d)+1)

        print gpm_zeit

        r_pro = 'ry'
        #try:


        r_pfad = ('/automount/radar/dwd/'+ r_pro +'/'+str(year)+'/'+str(year)+'-'+
                str(m)+'/'+ str(year)+'-'+str(m)+'-'+str(d)+'/raa01-'+r_pro+'_10000-'+
                str(ye)+str(m)+ str(d)+str(ht)+str(mt)+'-dwd---bi*')
        
        print r_pfad


        ## TRY: gibt es eine RADOLAN Datei zum Overpass!?
        ## read radolan ry file data and attributes
        try:
            rwdata, rwattrs = wradlib.io.read_radolan_composite(glob.glob(r_pfad)[0])

            radolan_grid_xy = wradlib.georef.get_radolan_grid(900,900)
            x = radolan_grid_xy[:,:,0]
            y = radolan_grid_xy[:,:,1]
            rwdata = np.ma.masked_equal(rwdata, -9999) *12 # Um auf mm/h zu kommen, beim Einlesen sind es mm/5min

            radolan_scan_time = rwattrs['datetime']
        except:
            print('__________________________________________RY File Missing!')
            
        else:    
            # copy data for binary grid
            bingrid = rwdata.copy()

            # Precipitation data in RADOLAN region
            # Posible precip products:
            
            # precipRateNearSurface
            dpr_pns = np.array(gpmdpr[sc]['SLV']['precipRateNearSurface'])
            dpr_pns[dpr_pns ==-9999.9]= np.nan
            # precipRateESurface
            #dpr_pes = np.array(gpmdpr[sc]['SLV']['precipRateESurface'])
            #dpr_pes[dpr_pes ==-9999.9]= np.nan
            # precipRateAve24
            dpr_pav = np.array(gpmdpr[sc]['SLV']['precipRateAve24'])
            dpr_pav[dpr_pav ==-9999.9]= np.nan
            # precip rate 3d
            dpr_p3d = np.array(gpmdpr[sc]['SLV']['precipRate'])
            dpr_p3d[dpr_p3d ==-9999.9]= np.nan

            # Brightband height
            dpr_bbh = np.array(gpmdpr[sc]['CSF']['heightBB'], dtype=float)
            dpr_bbh[dpr_bbh ==-9999.9] = np.nan

            # Bright-band width
            dpr_bbw = np.array(gpmdpr[sc]['CSF']['widthBB'], dtype=float)
            dpr_bbw[dpr_bbw ==-9999.9] = np.nan

            # PhaseNearSurface
            dpr_pha = np.array(gpmdpr[sc]['SLV']['phaseNearSurface'], dtype=int)
            #dpr_pha = dpr_pha/100
            #dpr_phase[dpr_phase==255]=np.nan

            #typePrecip
            dpr_typ = np.array(gpmdpr[sc]['CSF']['typePrecip'], dtype=float)

            #StromTopHeight
            dpr_sth = np.array(gpmdpr[sc]['PRE']['heightStormTop'], dtype=float)
            dpr_sth[dpr_sth ==-9999.9] = np.nan

            #Elevation
            dpr_ele = np.array(gpmdpr[sc]['PRE']['elevation'], dtype=float)
            dpr_ele[dpr_ele == -9999] = np.nan

            #Clutterfreebottom
            dpr_cfb_bin = np.array(gpmdpr[sc]['PRE']['binClutterFreeBottom'], dtype=float)
            dpr_cfb_bin[dpr_cfb_bin==-9999]=np.nan
            
            dpr_eof = np.array(gpmdpr[sc]['PRE']['ellipsoidBinOffset'], dtype=float)
            dpr_eof[dpr_eof==-9999]=np.nan
            
            dpr_lza = np.array(gpmdpr[sc]['PRE']['localZenithAngle'], dtype=float)
            dpr_lza[dpr_lza==-9999]=np.nan
            
            # Berechnung der clutterfree bottom height
            
            dpr_cfb = calc_height(dpr_cfb_bin, dpr_eof, dpr_lza, scan_swath=sc)
            
            # Landsurfacetyp
            dpr_lst = np.array(gpmdpr[sc]['PRE']['landSurfaceType'], dtype=float)
            dpr_lst[dpr_lst==-9999]=np.nan

            #3D Phase
            dpr_3dpha = np.array(gpmdpr[sc]['DSD']['phase'], dtype=int)
            #dpr_3dpha[dpr_3dpha == -9999] = np.nan
            
            # PiaFinal
            #dpr_pia = np.array(gpmdpr[sc]['SLV']['piaFinal'], dtype=float)
            #dpr_pia[dpr_pia == -9999] = np.nan
            
            # flagHeavyIcePrecip FLAG!
            # dpr_ice = np.array(gpmdpr[sc]['PRE']['flagHeavyIcePrecip'], dtype=float)
            # dpr_ice[dpr_ice == -99] = np.nan
            



            # GPM coordinate projection
            gpm_x, gpm_y = wradlib.georef.reproject(gpm_lon, gpm_lat,
                                                    projection_target=proj_stereo,
                                                    projection_source=proj_wgs)


            ## Remove all GPM footprint that are not in scan area of radolan
            ## -------------------------------------------------------------

            # GPM Koordinaten ravel
            gpm_xy = np.vstack((gpm_x.ravel(), gpm_y.ravel())).transpose()

            # index for GPM footprints in scan area
            oi_idx = idx_sr_in_gr_area(bingrid, gpm_xy)
            
            # check if idx is empty
            if len(oi_idx)==0:
                print ("____No Index of Overpass in Radolan region____")
                pass
            
            else:
                # entferne alle Indizes auserhalb des äußeren Polygons
                gpm_xy = gpm_xy[oi_idx]
                gpm_pns = dpr_pns.ravel()[oi_idx]
                #gpm_pes = dpr_pes.ravel()[oi_idx]
                gpm_pav = dpr_pav.ravel()[oi_idx]
                gpm_bbh = dpr_bbh.ravel()[oi_idx]
                gpm_bbw = dpr_bbw.ravel()[oi_idx]
                gpm_pha = dpr_pha.ravel()[oi_idx]
                gpm_typ = dpr_typ.ravel()[oi_idx]
                gpm_sth = dpr_sth.ravel()[oi_idx]
                gpm_ele = dpr_ele.ravel()[oi_idx]
                gpm_cfb = dpr_cfb.ravel()[oi_idx]
                gpm_lst = dpr_lst.ravel()[oi_idx]
                gpm_tim = gpm_scan_times.ravel()[oi_idx]
                # Parameters only for new pns product
                dpr_p3d = np.reshape(dpr_p3d,[dpr_pns.shape[0]*dpr_pns.shape[1],dpr_para[sc]])
                gpm_p3d = dpr_p3d[oi_idx,:]
                # 3d phase for adjusting
                dpr_3dpha = np.reshape(dpr_3dpha,[dpr_3dpha.shape[0]*dpr_3dpha.shape[1],dpr_para[sc]])
                gpm_ph3d = dpr_3dpha[oi_idx,:]
                
                gpm_cfb = dpr_cfb.ravel()[oi_idx]
                gpm_lza = dpr_lza.ravel()[oi_idx]
                gpm_eof = dpr_eof.ravel()[oi_idx]
                gpm_cfb_bin = dpr_cfb_bin.ravel()[oi_idx]


                ## Remove all RADOLAN Points not included in GPM DPR swath
                ## -------------------------------------------------------

                # Contour from ORIGINAL DPR Overpass
                dpr_contour_x = dpr_swath_contour(gpm_x)
                dpr_contour_y = dpr_swath_contour(gpm_y)
                print(dpr_contour_x.shape)

                # create dpr polygon of dpr xy contours 
                dpr_xy_poly = np.vstack((dpr_contour_x.ravel(), dpr_contour_y.ravel())).transpose()

                #load regular radolan grid and overlie with dpr contours
                zd_poly_radolan = wradlib.zonalstats.ZonalDataPoint(zd, [dpr_xy_poly], buf=2.5)

                # get radolan index in dpr swath
                ry_idx = zd_poly_radolan.get_source_index(0)

                # get radolan index in dpr swath AND in binary grid
                rwtest = ~np.ma.masked_less(rwdata, 0).mask

                radolan_idx = np.flatnonzero(rwtest.ravel())

                ry_idx2 = np.intersect1d(ry_idx, radolan_idx, assume_unique=True)

                # Radolan xy array
                xy_radolan = np.vstack((x.ravel(), y.ravel())).transpose()

                # Extract all affected RADOLAN grid points
                rwdata_cut = rwdata.ravel()[ry_idx2]
                x_cut = xy_radolan[...,0].ravel()[ry_idx2].copy()
                y_cut = xy_radolan[...,1].ravel()[ry_idx2].copy()


                # extrect all Height points (need to have the same shape as radolan grid)
                radolan_height_min_cut = rhmin.ravel()[ry_idx2]
                radolan_height_max_cut = rhmax.ravel()[ry_idx2]
                radolan_height_mean_cut = rhmean.ravel()[ry_idx2]
                radolan_height_std_cut = rhstd.ravel()[ry_idx2]


                # Compare all cutted radolan x and y 
                xy_cut = np.vstack((x_cut.ravel(), y_cut.ravel())).transpose()

                #Dpr footprint radius 
                dpr_footprint = 2.6 #radius in km

                # Interpolation with antena weighted function
                ry_pns_w = ipoli_radi_w(xy_cut, gpm_xy, rwdata_cut, dpr_footprint, k=25)

                # Interpolation of other parameters on dpr grid
                radolan_height_min_cut_ipoli = ipoli_radi(xy_cut, gpm_xy, radolan_height_min_cut, dpr_footprint, k=25, calc='min')
                radolan_height_max_cut_ipoli = ipoli_radi(xy_cut, gpm_xy, radolan_height_max_cut, dpr_footprint, k=25, calc='max')
                radolan_height_mean_cut_ipoli = ipoli_radi(xy_cut, gpm_xy, radolan_height_mean_cut, dpr_footprint, k=25)
                radolan_height_std_cut_ipoli = ipoli_radi(xy_cut, gpm_xy, radolan_height_std_cut, dpr_footprint, k=25)

                ## Create adjusted pns product from p3d
                ## -------------------------------

                # delta h calculation
                delta_h = radolan_height_min_cut_ipoli - gpm_cfb
                # new bin height  
                new_bin = calc_bin(gpm_cfb + delta_h, gpm_eof, gpm_lza)
                # round for bin
                new_bin = np.ceil(new_bin)
                # error calc with nan removal
                new_bin[new_bin<0]=np.nan

                # adjusted precip produkt 
                gpm_app = []
                
                # adjusted phase 
                gpm_aph = []
                    
                    
                for j in range(len(new_bin)):

                    if np.isnan(new_bin[j]):
                        # Check if bin is nan 
                        gpm_app.append(np.nan)
                        gpm_aph.append(np.nan)


                    else:  
                            if (new_bin[j]-1)>=dpr_para[sc]:
                            # Check if bin index bigger than array
                            # happen by round bin
                                gpm_app.append(np.nan)
                                gpm_aph.append(np.nan)
                            else:
                                gpm_app.append(gpm_p3d[j,int(new_bin[j])-1])
                                gpm_aph.append(gpm_ph3d[j,int(new_bin[j])-1])

                gpm_app = np.array(gpm_app)

                ## Write and save to HDF5 file
                ## ----------------------------
                name = '/automount/ags/velibor/gpmdata/dumpdataV7/' \
                                   'dpr_ry_'+sc+'/dprrado_'+ sc + str(gpm_zeit)
                # array with datetime in array with datimestrings
                gpm_tim_str = [dt.datetime.strftime(date, "%Y-%m-%d %H:%M:%S") for date in gpm_tim]    

                write2hdf(name, gpm_xy[:,0], gpm_xy[:,1],
                          gpm_pns, gpm_app, gpm_pav,
                          gpm_bbh, gpm_bbw, gpm_pha,
                          gpm_typ, gpm_sth, gpm_cfb,
                          gpm_lst, gpm_ele, gpm_tim_str,
                          ry_pns_w, xy_cut, rwdata_cut,
                          radolan_height_max_cut_ipoli,
                          radolan_height_min_cut_ipoli, 
                          radolan_height_mean_cut_ipoli,
                          radolan_height_std_cut_ipoli,
                          gpm_zeit,
                          dt.datetime.strftime(radolan_scan_time, "%Y-%m-%d %H:%M:%S"), gpm_aph)



2015.04.10 -- 03:24:40
2015.04.10 -- 03:24:40
/automount/radar/dwd/ry/2015/2015-04/2015-04-10/raa01-ry_10000-1504100325-dwd---bi*
(625,)
2015.04.09 -- 21:00:14
2015.04.09 -- 21:00:14
/automount/radar/dwd/ry/2015/2015-04/2015-04-09/raa01-ry_10000-1504092100-dwd---bi*
(387,)
2015.04.09 -- 04:17:48
2015.04.09 -- 04:17:48
/automount/radar/dwd/ry/2015/2015-04/2015-04-09/raa01-ry_10000-1504090415-dwd---bi*
(499,)
2015.04.08 -- 20:17:40
2015.04.08 -- 20:17:40
/automount/radar/dwd/ry/2015/2015-04/2015-04-08/raa01-ry_10000-1504082015-dwd---bi*
(501,)
2015.04.08 -- 03:35:11
2015.04.08 -- 03:35:11
/automount/radar/dwd/ry/2015/2015-04/2015-04-08/raa01-ry_10000-1504080335-dwd---bi*
(391,)
2015.04.07 -- 21:10:45
2015.04.07 -- 21:10:45
/automount/radar/dwd/ry/2015/2015-04/2015-04-07/raa01-ry_10000-1504072110-dwd---bi*
(623,)
2015.04.07 -- 04:28:27
2015.04.07 -- 04:28:27
/automount/radar/dwd/ry/2015/2015-04/2015-04-07/raa01-ry_10000-1504070430-dwd---bi*
(657,)
2015.04.06 -- 22:03:55
2015.04.06 -- 22:0

/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:260: RuntimeWarning: invalid value encountered in less


2014.12.15 -- 12:23:55
2014.12.15 -- 12:23:55
/automount/radar/dwd/ry/2014/2014-12/2014-12-15/raa01-ry_10000-1412151225-dwd---bi*
not relevant polygon removed
____No Index of Overpass in Radolan region____
2014.12.15 -- 05:59:25
2014.12.15 -- 05:59:25
/automount/radar/dwd/ry/2014/2014-12/2014-12-15/raa01-ry_10000-141215655-dwd---bi*
__________________________________________RY File Missing!
2014.12.14 -- 13:17:16
2014.12.14 -- 13:17:16
/automount/radar/dwd/ry/2014/2014-12/2014-12-14/raa01-ry_10000-1412141315-dwd---bi*
(507,)
2014.12.14 -- 06:53:03
2014.12.14 -- 06:53:03
/automount/radar/dwd/ry/2014/2014-12/2014-12-14/raa01-ry_10000-1412140655-dwd---bi*
(527,)
2014.12.13 -- 14:10:51
2014.12.13 -- 14:10:51
/automount/radar/dwd/ry/2014/2014-12/2014-12-13/raa01-ry_10000-1412131410-dwd---bi*
(623,)
2014.12.13 -- 07:46:22
2014.12.13 -- 07:46:22
/automount/radar/dwd/ry/2014/2014-12/2014-12-13/raa01-ry_10000-1412130745-dwd---bi*
____No Index of Overpass in Radolan region____
2014.12.13 -- 06:1

/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:40: RuntimeWarning: All-NaN slice encountered
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:31: RuntimeWarning: All-NaN slice encountered
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/ipykernel/__main__.py:22: RuntimeWarning: Mean of empty slice


2014.06.05 -- 21:42:41
2014.06.05 -- 21:42:41
/automount/radar/dwd/ry/2014/2014-06/2014-06-05/raa01-ry_10000-1406052140-dwd---bi*
(659,)
2014.06.05 -- 15:18:00
2014.06.05 -- 15:18:00
/automount/radar/dwd/ry/2014/2014-06/2014-06-05/raa01-ry_10000-1406051520-dwd---bi*
(305,)
2014.06.05 -- 13:42:15
2014.06.05 -- 13:42:15
/automount/radar/dwd/ry/2014/2014-06/2014-06-05/raa01-ry_10000-1406051340-dwd---bi*
____No Index of Overpass in Radolan region____


In [ ]:
"""import h5py
import numpy as np
import matplotlib.pyplot as pl
import matplotlib.pyplot as plt

import wradlib
from osgeo import osr
import datetime as dt
import glob
from osgeo import gdal
from scipy.spatial import cKDTree
import wradlib as wrl
import matplotlib as mpl

#% matplotlib inline 

import datetime as dt

pfad = glob.glob("/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS/dprrado*")

print ('Es gibt ' + str(len(pfad)) + ' Overpasses')

para_dict = {'sr_x': [],
             'sr_y': [],
             'gr_h_max': [],
             'gr_h_mean': [],
             'gr_h_min': [],
             'gr_h_std': [],
             ##!memory!##'gr_pp': [],
             'gr_pp_ipoli': [],
             ##### 'gr_time': np.array([]),
             ##!memory!##'gr_xy': [],
             'sr_bbh': [],
             'sr_bbw': [],
             'sr_cfb': [],
             'sr_ele': [],
             'sr_lst': [],
             'sr_pav': [],
             'sr_pes': [],
             'sr_phase': [],
             'sr_pns': [],
             'sr_sth': [],
             ##### 'sr_time': np.array([]),
             'sr_times': [],
             'sr_type': []
}   

a=0

for dict_name in para_dict.keys():
    # DR als liste
    print(dict_name)
    
    if 'gr' in dict_name:
        gr_name = 'GR'
    else:
        gr_name = 'SR'
    
    for p in pfad:

        DR = h5py.File(p, 'r')
        # for über liste
        a+=1
        # Listen append x.append(x)
        para_dict[dict_name].append(DR[gr_name][dict_name][:])
        
        # ! Colse hdf5 file ! 
        DR.close()
        del(DR)
        
        
    ac = np.concatenate(para_dict[dict_name],axis=0)
    print (ac.shape)

    np.save('/automount/ftp/velibor/data/'+dict_name+'.npy',ac) 
    para_dict.pop(dict_name,None)"""

In [16]:
import h5py
import numpy as np
import matplotlib.pyplot as pl
import matplotlib.pyplot as plt

import wradlib
from osgeo import osr
import datetime as dt
import glob
from osgeo import gdal
from scipy.spatial import cKDTree
import wradlib as wrl
import matplotlib as mpl

#% matplotlib inline 

###### TODO calc also Timedifferenze!!!!!!!!!!! ##############

import datetime as dt

pfad = glob.glob("/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS/dprrado*")

print ('Es gibt ' + str(len(pfad)) + ' Overpasses')

para_dict = {'sr_x': [],
             'sr_y': [],
             'sr_aph': [],
             'gr_h_max': [],
             'gr_h_mean': [],
             'gr_h_min': [],
             'gr_h_std': [],
             ##!memory!##'gr_pp': [],
             'gr_pp_ipoli': [],
             ##### 'gr_time': np.array([]),
             ##!memory!##'gr_xy': [],
             'sr_bbh': [],
             'sr_bbw': [],
             'sr_cfb': [],
             'sr_ele': [],
             'sr_lst': [],
             'sr_pav': [],
             'sr_pes': [],
             'sr_phase': [],
             'sr_pns': [],
             'sr_sth': [],
             ##### 'sr_time': np.array([]),
             'sr_times': [],
             'sr_type': []
}   

a=0

for dict_name in para_dict.keys():
    # DR als liste
    print(dict_name)
    
    if 'gr' in dict_name:
        gr_name = 'GR'
    else:
        gr_name = 'SR'
    
    for p in pfad:

        DR = h5py.File(p, 'r')
        # for über liste
        a+=1
        # Listen append x.append(x)
        para_dict[dict_name].append(DR[gr_name][dict_name][:])
        
        # ! Colse hdf5 file ! 
        DR.close()
        del(DR)
        
        
    ac = np.concatenate(para_dict[dict_name],axis=0)
    print (ac.shape)

    np.save('/automount/ftp/velibor/data/'+dict_name+'.npy',ac) 
    para_dict.pop(dict_name,None)

Es gibt 2611 Overpasses
sr_times
(12143131,)
sr_ele
(12143131,)
sr_cfb
(12143131,)
gr_h_std
(12143131,)
sr_pns
(12143131,)
sr_x
(12143131,)
sr_y
(12143131,)
gr_h_mean
(12143131,)
sr_aph
(12143131,)
sr_bbw
(12143131,)
sr_bbh
(12143131,)
sr_phase
(12143131,)
gr_h_min
(12143131,)
sr_pav
(12143131,)
sr_type
(12143131,)
sr_pes
(12143131,)
sr_lst
(12143131,)
gr_pp_ipoli
(12143131,)
gr_h_max
(12143131,)
sr_sth
(12143131,)


In [105]:
import h5py
import numpy as np
import matplotlib.pyplot as pl
import matplotlib.pyplot as plt

import wradlib
from osgeo import osr
import datetime as dt
import glob
from osgeo import gdal
from scipy.spatial import cKDTree
import wradlib as wrl
import matplotlib as mpl

#% matplotlib inline 

###### TODO calc also Timedifferenze!!!!!!!!!!! ##############

import datetime as dt

pfad = glob.glob("/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS/dprrado*")

print ('Es gibt ' + str(len(pfad)) + ' Overpasses')

para_dict = {'sr_times': []}   # 'gr_time': np.array([]),

a=0

for p in sorted(pfad):

    DR = h5py.File(p, 'r')
    # for über liste
    a+=1
    # Listen append x.append(x)
    T1 = np.array(DR['SR']['sr_times'][:], dtype='datetime64')
    T2 = np.array(DR['GR']['gr_time'].value, dtype='datetime64' )
                
    para_dict['sr_times'].append( T1 - T2 )

    # ! Colse hdf5 file ! 
    DR.close()
    del(DR)
        
        
    ac = np.concatenate(para_dict['sr_times'],axis=0)
    #print (ac.shape)

    np.save('/automount/ftp/velibor/data/'+'timedelta'+'.npy',ac) 
    #para_dict.pop(dict_name,None)

Es gibt 2611 Overpasses
(5332,)
(8622,)
(9884,)
(17229,)
(25658,)
(26390,)
(34547,)
(39463,)
(43767,)
(50715,)
(51871,)
(52235,)
(52999,)
(55756,)
(62197,)
(64848,)
(71086,)
(80068,)
(80102,)
(80256,)
(88981,)
(95029,)
(98540,)
(105546,)
(108084,)
(108528,)
(115582,)
(123874,)
(124854,)
(128245,)
(134179,)
(136244,)
(142988,)
(152021,)
(152043,)
(152653,)
(160843,)
(165528,)
(173661,)
(174807,)
(174827,)
(182362,)
(184346,)
(189544,)
(193768,)
(194874,)
(202474,)
(211232,)
(211592,)
(213440,)
(220154,)
(223473,)
(228798,)
(228863,)
(237666,)
(243831,)
(247281,)
(254375,)
(256740,)
(257084,)
(265528,)
(273677,)
(274835,)
(278560,)
(284269,)
(286186,)
(293027,)
(302053,)
(302082,)
(302727,)
(310894,)
(315765,)
(320446,)
(328545,)
(329752,)
(329788,)
(337150,)
(344792,)
(346606,)
(351547,)
(356215,)
(357474,)
(364902,)
(373761,)
(373991,)
(375490,)
(382885,)
(386629,)
(392120,)
(400858,)
(401181,)
(409934,)
(416670,)
(419531,)
(425703,)
(428770,)
(429333,)
(437128,)
(445387,)
(446209,)
(4

In [80]:
DR['GR']['gr_time'].value

'2014-03-19 13:00:00'

In [98]:
a = np.array(DR['GR']['gr_time'].value, dtype='datetime64' )

In [96]:
a = DR['GR']['gr_time'].value

In [99]:
T1[0] - a

numpy.timedelta64(18,'s')

In [103]:
para_dict = {'sr_times': []}

In [104]:
para_dict['sr_times']

[]

In [44]:
p =pfad[10]

DR = h5py.File(p, 'r')
# for über liste
a+=1
# Listen append x.append(x)
T1 = np.array(DR['SR']['sr_times'][:], dtype='datetime64')
T2 = np.array(DR['GR']['gr_time'], dtype='datetime64' )

para_dict['sr_times'].append( T1 - T2 )

# ! Colse hdf5 file ! 
DR.close()
del(DR)


ac = np.concatenate(para_dict['sr_times'],axis=0)


TypeError: No conversion path for dtype: dtype('<M8')

In [51]:
DR['GR']['gr_time']

<HDF5 dataset "gr_time": shape (), type "|O">

# 

In [42]:
pfad[10]


'/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS/dprrado_NS2014.07.18 -- 09:24:14.hdf5'

In [ ]:
gpm_dir = gpm_dir[1202:-1]


In [ ]:
len(gpm_dir)

In [ ]:
gpm_dir[0]

In [108]:
1500/60.

25.0